In [1]:
from Data_Processor import Data_Processor
from Data_Processor import clean
import numpy as np
from matplotlib import pyplot as plt

[nltk_data] Downloading package wordnet to /Users/zhuying/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zhuying/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhuying/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
DP=Data_Processor(start_month='2017-06',end_month='2020-05',
                  template=["BankofAmerica"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
noise=DP.removenoise() 
DP.clean()
print(DP.datanums())

([10064, 10245, 9899, 10158, 10087, 9884, 10173, 9714, 10210, 9907, 9835, 10009, 9974, 9930, 10206, 10125, 9922, 9918, 10087, 9530, 10053, 9989, 9987, 8677, 8860, 9688, 9877, 10080, 10040, 9894, 10145, 9899, 10176, 10137, 10248, 10011], 357638)
([4715, 4056, 4022, 4396, 4725, 4957, 4980, 4692, 4931, 4655, 5433, 5753, 6230, 5422, 5751, 5824, 5585, 5762, 6451, 5821, 5848, 6737, 6409, 5910, 6503, 6783, 7050, 7384, 7123, 7570, 7445, 6612, 6631, 7319, 8145, 7221], 214851)


In [3]:
DP.textdata()['2020-05'][:10]

[' Chief Executive Officer Bank of America\n Hon Chairman Royal DSM Special Envoy on Coronavirus Dutch Government\n Chairman Siemens and Maersk',
 'I have a contact for BofA in the San Antonio area if youd like it \n\nIm in Las Cruces NM but our market was in El Paso when i worked for them So Im familiar with them all over there Same region\n\nLet me know',
 'I did get to finish the Intralinks portion for BankofAmerica We also applied with bluevine paypal kabbage fundera crossriverbank Lendio FundingCircleUS so but not hopeful',
 'Ive got  people Im willing through the loan mess at Bank of America\n\nSome bankers are helpful some literally should get bumped down to janitor and I dont even know about that Keys are sharp They can get hurt\n\nIf you need help with follow and ',
 'Also the BankofAmerica representative I spoke with today could not explain what happened And told me that the Fraud department had gone home for the day',
 'My biggest returns so far even though very little are d

In [10]:
from pycorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP('http://localhost:9000') #the port here should be the same as above (openned server)
#nlp.annotate will return a dictionary (key is 'sentences')
res={}
for i in range(len(DP.textdata()['2020-05'])):
    sentence = DP.textdata()['2020-05'][i]


    #here nlp.annotate process each sentence in the paragraph above using the annotators specified below
    res[i] = nlp.annotate(sentence,
                       properties={
                           'annotators': 'tokenize,ssplit,pos,parse,sentiment',
                           'outputFormat': 'json',
                           'timeout': 100000,
                       })

In [7]:
for i in range(10):
    for s in res[i]["sentences"]:
        print("%d: '%s': %s %s" % (
            s["index"],
            " ".join([t["word"] for t in s["tokens"]]),
            s["sentimentValue"], s["sentiment"]))

0: 'Chief Executive Officer Bank of America Hon Chairman Royal DSM Special Envoy on Coronavirus Dutch Government Chairman Siemens and Maersk': 2 Neutral
0: 'I have a contact for BofA in the San Antonio area if youd like it Im in Las Cruces NM but our market was in El Paso when i worked for them So Im familiar with them all over there Same region Let me know': 2 Neutral
0: 'I did get to finish the Intralinks portion for BankofAmerica We also applied with bluevine paypal kabbage fundera crossriverbank Lendio FundingCircleUS so but not hopeful': 1 Negative
0: 'Ive got people Im willing through the loan mess at Bank of America Some bankers are helpful some literally should get bumped down to janitor and I do nt even know about that Keys are sharp They can get hurt If you need help with follow and': 1 Negative
0: 'Also the BankofAmerica representative I spoke with today could not explain what happened And told me that the Fraud department had gone home for the day': 2 Neutral
0: 'My biggest

In [16]:
import pandas as pd
result = pd.DataFrame(columns=['text', 'sentimentValue', 'sentiment'])
for i in range(len(DP.textdata()['2020-05'])):
    for s in res[i]["sentences"]:
        '''
        print("%d: '%s': %s %s" % (
            s["index"],
            " ".join([t["word"] for t in s["tokens"]]),
            s["sentimentValue"], s["sentiment"]))
        '''
        result = result.append(pd.DataFrame({'text':[" ".join([t["word"] for t in s["tokens"]])],'sentimentValue':[s["sentimentValue"]],'sentiment':[s["sentiment"]]}),ignore_index=True)  

In [17]:
result

,text,sentimentValue,sentiment
0,Chief Executive Officer Bank of America Hon Ch...,2,Neutral
1,I have a contact for BofA in the San Antonio a...,2,Neutral
2,I did get to finish the Intralinks portion for...,1,Negative
3,Ive got people Im willing through the loan mes...,1,Negative
4,Also the BankofAmerica representative I spoke ...,2,Neutral
...,...,...,...
7215,Im listening to family member on the phone dea...,2,Neutral
7216,Thanks to ongoing support from BankofAmerica V...,2,Neutral
7217,Thread cont We were one of the st applicants i...,1,Negative
7218,BankofAmerica how is it possible to change cas...,3,Positive


In [12]:
pd.set_option('display.max_colwidth', -1)

In [13]:
df[['text','label','score']]

,text,label,score
0,Chief Executive Officer Bank of America\n Hon Chairman Royal DSM Special Envoy on Coronavirus Dutch Government\n Chairman Siemens and Maersk,NEUTRAL,0.718565
1,I have a contact for BofA in the San Antonio area if youd like it \n\nIm in Las Cruces NM but our market was in El Paso when i worked for them So Im familiar with them all over there Same region\n\nLet me know,POSITIVE,0.826208
2,I did get to finish the Intralinks portion for BankofAmerica We also applied with bluevine paypal kabbage fundera crossriverbank Lendio FundingCircleUS so but not hopeful,NEUTRAL,0.487073
3,Ive got people Im willing through the loan mess at Bank of America\n\nSome bankers are helpful some literally should get bumped down to janitor and I dont even know about that Keys are sharp They can get hurt\n\nIf you need help with follow and,NEGATIVE,0.125745
4,Also the BankofAmerica representative I spoke with today could not explain what happened And told me that the Fraud department had gone home for the day,NEGATIVE,0.180704
...,...,...,...
7182,Im listening to family member on the phone dealing with BankofAmerica service trying to fix an error on their part\n\nIts been over years since Ive banked with I forgot how awful that bank is\n\n,NEGATIVE,0.055388
7183,Thanks to ongoing support from BankofAmerica VetslegalOC can advance our mission to provide crucial services for veterans to provide free legal services to remove barriers to their housing employment and healthcare as we navigate the coronavirus crisis,POSITIVE,0.888642
7184,Thread cont\n\n We were one of the st applicants in the country complied with all documentation\n BofA does not have knowledgeable assistants\n We then fulfilled their request for additional documents perfectly\n Now they blame SBA SBA blames BofA\nRT Thread,NEUTRAL,0.643762
7185,BankofAmerica how is it possible to change cash rewards category website page nonfunctional for weeks,NEUTRAL,0.641832


In [28]:
list(result[result["sentiment"]=='Negative']['text'])

['I did get to finish the Intralinks portion for BankofAmerica We also applied with bluevine paypal kabbage fundera crossriverbank Lendio FundingCircleUS so but not hopeful',
 'Ive got people Im willing through the loan mess at Bank of America Some bankers are helpful some literally should get bumped down to janitor and I do nt even know about that Keys are sharp They can get hurt If you need help with follow and',
 'BankofAmerica is trying to take my home of years as I am mo behind on mortgage Yet in BofACorp was rescued by the US govt thru BULLON bailout a guarantee for almost B of potential losses on toxic assets RepKatiePorter SpeakerPelosi',
 'I am not aware of it exactlymy branch has been moved from our place to new place',
 'Yeah if you wan na drop a bill or two you can get down low Im cheap I always sit up top Not a bad seat there or at Bank of America',
 'I did get to finish the Intralinks portion for BankofAmerica We also applied with bluevine paypal kabbage fundera crossrive

In [26]:
result[result["sentiment"]=='Positive']

,text,sentimentValue,sentiment
5,My biggest returns so far even though very lit...,3,Positive
6,I called early last week to report a fraudulen...,3,Positive
8,Oh yeah they moved before then Its right down ...,3,Positive
11,That really helps a lot Thank you so much for ...,3,Positive
19,My biggest returns so far even though very lit...,3,Positive
...,...,...,...
7203,Yea pho next to Bank of America is good,3,Positive
7205,Support the Kearney Center celebrates global u...,3,Positive
7206,The fact that there is nobody to contact regar...,3,Positive
7214,Thread We are almost one month from our origin...,3,Positive


In [27]:
result[result["sentiment"]=='Negative']

,text,sentimentValue,sentiment
2,I did get to finish the Intralinks portion for...,1,Negative
3,Ive got people Im willing through the loan mes...,1,Negative
7,BankofAmerica is trying to take my home of yea...,1,Negative
9,I am not aware of it exactlymy branch has been...,1,Negative
13,Yeah if you wan na drop a bill or two you can ...,1,Negative
...,...,...,...
7201,Thread BankofAmerica has been a living nightma...,1,Negative
7208,Thread We have honorably retained paid workers...,1,Negative
7211,BankofAmerica I disputed a claim and BOA refun...,1,Negative
7217,Thread cont We were one of the st applicants i...,1,Negative


In [29]:
result[result["sentiment"]=='Neutral']

,text,sentimentValue,sentiment
0,Chief Executive Officer Bank of America Hon Ch...,2,Neutral
1,I have a contact for BofA in the San Antonio a...,2,Neutral
4,Also the BankofAmerica representative I spoke ...,2,Neutral
10,No problem,2,Neutral
12,Follow me and Ill DM you some info on some Bof...,2,Neutral
...,...,...,...
7210,Berkeshire Hathaways Top Holding by Value B B ...,2,Neutral
7212,Bank stocks could surge in economic recovery B...,2,Neutral
7213,Got my Trump Bucks in the bank and ready to do...,2,Neutral
7215,Im listening to family member on the phone dea...,2,Neutral


In [4]:
import json
import csv 
  
with open('2020-05.json', 'r') as f:
    data = json.load(f)

data_file = open('data_file.csv', 'w') 
  
# create the csv writer object 
csv_writer = csv.writer(data_file) 
  
# Counter variable used for writing  
# headers to the CSV file 
count = 0
  
for emp in data: 
    if count == 0: 
  
        # Writing headers of CSV file 
        header = emp.keys() 
        csv_writer.writerow(header) 
        count += 1
  
    # Writing data of CSV file 
    csv_writer.writerow(emp.values()) 
  
data_file.close()

In [2]:
dl = datelist('2020-05', '2020-05')

In [3]:
dl

['2020-05']

In [5]:
import pandas as pd
df=pd.read_csv('data_file.csv')

In [10]:
df[['text','label','score']]

,text,label,score
0,Chief Executive Officer Bank of America\n Hon...,NEUTRAL,0.718565
1,I have a contact for BofA in the San Antonio a...,POSITIVE,0.826208
2,I did get to finish the Intralinks portion for...,NEUTRAL,0.487073
3,Ive got people Im willing through the loan me...,NEGATIVE,0.125745
4,Also the BankofAmerica representative I spoke ...,NEGATIVE,0.180704
...,...,...,...
7182,Im listening to family member on the phone dea...,NEGATIVE,0.055388
7183,Thanks to ongoing support from BankofAmerica V...,POSITIVE,0.888642
7184,Thread cont\n\n We were one of the st applica...,NEUTRAL,0.643762
7185,BankofAmerica how is it possible to change c...,NEUTRAL,0.641832
